<a href="https://colab.research.google.com/github/Tania-Ari/Programacion_I/blob/main/Proyecto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from google.colab import files

class Excel:
    """Esta clase lo que hace es cargar un archivo de tipo .xlsx y manipularlo"""
    def __init__(self, archivo_excel):
        """Es la funcion constructora, que inicializa la clase
        Usa como parametros un archivo excel"""
        self.df = pd.read_excel(archivo_excel)  # Carga el archivo Excel y guarda los datos en self.df
        self.dic = {}  # Inicializa el diccionario dic
        self.sub = {}  # Inicializa el diccionario sub
        self.arcos = {}  # Inicializa el diccionario arcos
        self.duraciones = {}  # Nuevo atributo para almacenar las duraciones


    def llenar_diccionarios(self):
        """Funcion que llena los diccionarios dic y sub con los datos obtenidos
         en el documento xlsx """
        for index, row in self.df.iterrows():  # Itera sobre las filas
            actividad = str(row['Actividad'])  # Obtiene la actividad de la fila y la convierte en cadena
            precedentes = row['Precedentes']  # Obtiene los precedentes de la fila
            duracion = row['Duracion']  # Nuevo: extraer duración de la columna Duración



            if pd.notna(precedentes):  # Verifica que los precedentes no sean NaN
                precedentes = str(precedentes).split(', ')  # Convierte los precedentes a lista
            else:
                precedentes = []  # Si son NaN, asigna una lista vacía

            self.dic[actividad] = precedentes  # Agrega la actividad y sus precedentes al diccionario dic
            self.duraciones[actividad] = duracion  # Almacenar la duración

    def actividades_completadas(self):
        """Recorre el diccionario dic y encuentra las actividades completas
        y devuelve una lista de actividades completas (complete_activities)"""
        complete_activities = []  # Inicializa una lista para actividades completadas
        for actividad, precedentes in self.dic.items():  # Itera sobre el diccionario dic
            if not precedentes:  # Si la lista de precedentes está vacía
                complete_activities.append(actividad)  # Agrega la actividad a la lista de completas
        return complete_activities  # Devuelve la lista de actividades completas

    def reorganizar_diccionario(self):
        """vuelve a organizar la lista dic y la guarda en otra llamada organizado_dic"""
        complete_activities = self.actividades_completadas()  # Obtiene actividades completas
        while len(complete_activities) < len(self.dic):  # Mientras no todas las actividades estén completas
            for actividad, precedentes in self.dic.items():  # Itera sobre el diccionario dic
                if actividad not in complete_activities:  # Si la actividad no está en completas
                    if all(pre in complete_activities for pre in precedentes):  # Si todos los precedentes están en completas
                        complete_activities.append(actividad)  # Agrega la actividad a completas

        self.organizado_dic = {act: self.dic[act] for act in complete_activities}  # Crea un diccionario organizado

    def generar_grafica(self):
        """Genera una grafica a partir del diccionario organizado"""
        for a, ps in self.organizado_dic.items():  # Itera sobre el diccionario organizado
            self.sub[a] = []  # Asigna una lista vacía al subdiccionario para cada actividad
            for p in ps:
                self.sub[p].append(a)  # Agrega la actividad como precedente a las actividades relacionadas

        nodes = [0, 1]  # Inicializa los nodos del grafo
        done = [[], []]  # Inicializa el registro de actividades procesadas
        ind = 2  # Inicializa el índice para identificadores de nodos
        arrows = []  # Inicializa la lista de conexiones entre nodos

        for k, v in self.organizado_dic.items():  # Itera sobre el diccionario organizado
            if v == []:  # Si una actividad no tiene precedentes
                nodes.append(ind)  # Agrega un nuevo nodo al grafo
                arrows.append((0, ind))  # Conecta el nuevo nodo con el nodo inicial
                done.append([k])  # Registra la actividad como procesada
                self.arcos[k] = (0, ind)  # Registra la conexión entre la actividad y el nuevo nodo
                ind += 1  # Incrementa el índice para el siguiente nodo
            elif v not in done:  # Si los precedentes no han sido procesados
                separated = [[elem] for elem in v]  # Separa los precedentes en sublistas individuales
                for elem in separated:
                    arrows.append((done.index(elem), ind))  # Conecta el nodo de precedentes con un nuevo nodo
                done.append(v)  # Registra los precedentes como procesados
                nodes.append(ind)  # Agrega un nuevo nodo al grafo
                self.arcos[k] = (done.index(v), ind)  # Registra la conexión entre la actividad y el nuevo nodo
                ind += 1  # Incrementa el índice para el siguiente nodo
                if self.sub[k] == []:  # Si la actividad no tiene subactividades
                    arrows.append((done.index(v), 1))  # Conecta el nodo de precedentes con el nodo inicial
                else:  # Si la actividad tiene subactividades
                    nodes.append(ind)  # Agrega un nuevo nodo al grafo
                    arrows.append((done.index(v), ind))  # Conecta el nodo de precedentes con el nuevo nodo
                    done.append([k])  # Registra la actividad como procesada
                    self.arcos[k] = (done.index(v), ind)  # Registra la conexión entre la actividad y el nuevo nodo
                    ind += 1  # Incrementa el índice para el siguiente nodo
            else:  # Si los precedentes ya han sido procesados
                if self.sub[k] == []:  # Si la actividad no tiene actividades subsecuentes
                    arrows.append((done.index(v), 1))  # Conecta el nodo de precedentes con el nodo inicial
                    self.arcos[k] = (done.index(v), 1)  # Registra la conexión entre la actividad y el nodo inicial
                else:  # Si la actividad tiene actividades subsecuentes
                    nodes.append(ind)  # Agrega un nuevo nodo al grafo
                    arrows.append((done.index(v), ind))  # Conecta el nodo de precedentes con el nuevo nodo
                    done.append([k])  # Registra la actividad como procesada
                    self.arcos[k] = (done.index(v), ind)  # Registra la conexión entre la actividad y el nuevo nodo
                    ind += 1  # Incrementa el índice para el siguiente nodo

        #print("Nodos:", nodes)  # Imprime los nodos de la grafica
        #print("Arcos:", arrows)  # Imprime las conexiones entre nodos
        #print("actividad y su Arco:", self.arcos)
        #print(self.duraciones)
        #print(self.dic)
        #print(self.organizado_dic)
        #print(self.sub)

        # Aquí termina la generación de la gráfica
        # Llama al método para generar el reporte
        reporte = Reporte(self, self.duraciones)
        reporte.asignar_pesos()
        reporte.generar_reporte()



# Clase Reporte
class Reporte:
    """Esta clase genra un reporte con los datos del excel, en donde nos dice las
    actividades criticas y la duracion del proyecto """
    def __init__(self, grafica, duraciones):
        """Este es la funcion constructor de la clase reporte, recibe los nodos
        y nodos y tiene un diccionario con duraciones de cada actividad"""
        # Constructor de la clase Reporte
        self.grafica = grafica  # Asigna el argumento grafica al atributo grafica de la instancia
        self.duraciones = duraciones  # Asigna el argumento duraciones al atributo duraciones de la instancia

    def asignar_pesos(self):
        """Asigna pesos es decir la duracion a cada arco (actividad)"""
        # Método para asignar pesos a los arcos del grafo
        for actividad, arco in self.grafica.arcos.items():
            if arco[1] != 1:
                # Si el segundo elemento de la tupla arco no es igual a 1
                duracion_actividad = self.duraciones[actividad]  # Obtiene la duración de la actividad del diccionario duraciones
                # Actualiza el valor del arco en el diccionario arcos de grafica con la duración de la actividad
                self.grafica.arcos[actividad] = (arco[0], arco[1], duracion_actividad)

    def fechas_proximas_lejanas(self):
        """Calcula las fechas más próximas y lejanas para cada actividad en la gráfica."""
        # Método para calcular las fechas más próximas y lejanas para cada actividad
        fechas_proximas = {}
        fechas_lejanas = {}
        for actividad, arco in self.grafica.arcos.items():
            nodo_inicial = arco[0]
            # Obtiene la duración de la actividad si existe, de lo contrario, asigna 0
            duracion_actividad = arco[2] if len(arco) == 3 else 0

            # Calcula la fecha más próxima para la actividad actual
            fecha_proxima = duracion_actividad + max(fechas_proximas[pre] for pre in self.grafica.organizado_dic[actividad]) if self.grafica.organizado_dic[actividad] else 0
            # Calcula la fecha más lejana para la actividad actual
            fecha_lejana = duracion_actividad + min(fechas_lejanas[pre] for pre in self.grafica.organizado_dic[actividad]) if self.grafica.organizado_dic[actividad] else 0

            # Actualiza las fechas próximas y lejanas para la actividad actual
            if actividad not in fechas_proximas or fecha_proxima > fechas_proximas[actividad]:
                fechas_proximas[actividad] = fecha_proxima
            if actividad not in fechas_lejanas or fecha_lejana < fechas_lejanas[actividad]:
                fechas_lejanas[actividad] = fecha_lejana

        return fechas_proximas, fechas_lejanas

    def generar_reporte(self):
        """Genera el reporte del proyecto con la duracion minima y las actividades críticas."""
        # Método para generar el reporte con las fechas próximas y lejanas, y las actividades críticas
        fechas_proximas, fechas_lejanas = self.fechas_proximas_lejanas()
        # Obtiene las actividades críticas comparando las fechas próximas y lejanas
        actividades_criticas = [act for act, fecha_proxima in fechas_proximas.items() if fecha_proxima == fechas_lejanas[act]]
        # Obtiene la duración mínima necesaria para el proyecto
        duracion_minima = max(fechas_lejanas.values(), default=0)

        with open('reporte.txt', 'w') as f:
            # Escribe en el archivo la duración mínima necesaria para el proyecto
            f.write(f'Tiempo mínimo necesario para la elaboración del proyecto: {duracion_minima} semanas\n')
            f.write('Actividades críticas:\n')
            # Escribe cada actividad crítica en el archivo
            for act in actividades_criticas:
                f.write(f'- {act}\n')

        # Descarga el archivo 'reporte.txt'
        files.download('reporte.txt')


def main():
  # Llamada a las clases y ejecución del codigo
  archivo_excel = '/content/Creacion_de_la_carrera_de_mat_ap.xlsx'
  organizador = Excel(archivo_excel)  # Crear una instancia de la clase Excel con el archivo
  organizador.llenar_diccionarios()  # Llenar los diccionarios con datos del archivo Excel
  organizador.reorganizar_diccionario()  # Reorganizar el diccionario según algún criterio
  organizador.generar_grafica()  # Generar la gráfica utilizando los datos del diccionario


if __name__ == "__main__":
    main()




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>